In [ ]:

import os
import torch
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig

def init_distributed(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12345"
    if rank == 0:
        print("Initializing distributed process group...")
    torch.distributed.init_process_group(backend='nccl', world_size=world_size, rank=rank)

def cleanup_distributed():
    torch.distributed.destroy_process_group()

def main_worker(rank, world_size):
    init_distributed(rank, world_size)

    # Move the finetune() function here
    # Load the dataset
    dataset_name = "ruslanmv/ai-medical-dataset"
    dataset = load_dataset(dataset_name, split="train")
    # Select the first 1M rows of the dataset
    dataset = dataset.select(range(100))
    # Load the model + tokenizer
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.float16,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        trust_remote_code=True,
        use_cache=False,
    )
    # Replace the DDP wrapping part with the following lines
    model = model.to(rank)
    model = DDP(model, device_ids=[rank], output_device=rank)

    # PEFT config
    lora_alpha = 16
    lora_dropout = 0.1
    lora_r = 32  # 64
    peft_config = LoraConfig(
      lora_alpha=lora_alpha,
      lora_dropout=lora_dropout,
      r=lora_r,
      bias="none",
      task_type="CAUSAL_LM",
      target_modules=["k_proj", "q_proj", "v_proj", "up_proj", "down_proj", "gate_proj"],
      modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm", "norm"],
    )
    # Args
    max_seq_length = 512
    output_dir = "./results"
    per_device_train_batch_size = 2  # reduced batch size to avoid OOM
    gradient_accumulation_steps = 2
    optim = "adamw_torch"
    save_steps = 10
    logging_steps = 1
    learning_rate = 2e-4
    max_grad_norm = 0.3
    max_steps = 1  # 300 Approx the size of guanaco at bs 8, ga 2, 2 GPUs.
    warmup_ratio = 0.1
    lr_scheduler_type = "cosine"

    training_arguments = TrainingArguments(
      output_dir=output_dir,
      per_device_train_batch_size=per_device_train_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      optim=optim,
      save_steps=save_steps,
      logging_steps=logging_steps,
      learning_rate=learning_rate,
      fp16=True,
      max_grad_norm=max_grad_norm,
      max_steps=max_steps,
      warmup_ratio=warmup_ratio,
      group_by_length=True,
      lr_scheduler_type=lr_scheduler_type,
      gradient_checkpointing=True,  # gradient checkpointing
      report_to="wandb",
    )
    # Trainer
    trainer = SFTTrainer(
      model=model,
      train_dataset=dataset,
      peft_config=peft_config,
      dataset_text_field="context",
      max_seq_length=max_seq_length,
      tokenizer=tokenizer,
      args=training_arguments,
    )
    # Train :)
    trainer.train()
    cleanup_distributed()

if __name__ == "__main__":
    world_size = torch.cuda.device_count()

    processes = []
    for rank in range(world_size):
        p = mp.Process(target=main_worker, args=(rank, world_size))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()
